0. Read fundamental matrix (F) from the csv file.

1. Center data (choose any (or first one) to be the world center coordinate system)
- subtract translation vector to everything
- compute inverse of rotation matrix (then apply to every)

2. Apply assignment 2

In [7]:
from Dino_Images import Dino_Images

import numpy as np

DI = Dino_Images()

num_images, image_path, \
               centered_calibration_matrices, centered_rotation_matrices, centered_translation_vectors = DI.parse_par("/dinoSR_par.txt")


In [ ]:
# Select normalized coordinates for matched features that are inliers for essential matrix E. 
n_ptsL_in = n_ptsL[ind[E_inliers]]
n_ptsR_in = n_ptsR[ind[E_inliers]]
# Form matrix A in equation AX=0 where X represent 4 vectors (homogeneous representation of 3D point).
def get_matrix_A(pair_points, P_1, P_2):
    # X = np.append([n_ptsL, n_ptsR])
    A_row_1 = P_1[0, :] - pair_points[0]*P_1[2, :]
    A_row_2 = P_1[1, :] - pair_points[1]*P_1[2, :]
    A_row_3 = P_2[0, :] - pair_points[2]*P_2[2, :]
    A_row_4 = P_2[1, :] - pair_points[3]*P_2[2, :]
    
    A = np.vstack([A_row_1, A_row_2, A_row_3, A_row_4])
    
    return A

# Use your solution for Problem 6.
# Each camera (projection matrix P) will define its own A  

# HINT: to keep it simple, first solve the problem for one match.

Aa = np.apply_along_axis(get_matrix_A, 1, np.hstack([n_ptsL_in, n_ptsR_in]), Pw, Pa)
Ab = np.apply_along_axis(get_matrix_A, 1, np.hstack([n_ptsL_in, n_ptsR_in]), Pw, Pb)
Ac = np.apply_along_axis(get_matrix_A, 1, np.hstack([n_ptsL_in, n_ptsR_in]), Pw, Pc)
Ad = np.apply_along_axis(get_matrix_A, 1, np.hstack([n_ptsL_in, n_ptsR_in]), Pw, Pd)

In [ ]:
%matplotlib notebook

import numpy as np
import numpy.linalg as la
import matplotlib
import matplotlib.image as image
import matplotlib.pyplot as plt
from skimage.feature import (corner_harris, corner_peaks, plot_matches, BRIEF, match_descriptors)
from skimage.transform import warp, ProjectiveTransform, EssentialMatrixTransform, FundamentalMatrixTransform
from skimage.color import rgb2gray
from skimage.measure import ransac

# Indicate (E) inlier matches in image 1 and image 2
# loading two images (two camera views) and the corresponding matrix K (intrinsic parameters)
imL = image.imread("images/kronan1.jpg")
imR = image.imread("images/kronan2.jpg")
imLgray = rgb2gray(imL)
imRgray = rgb2gray(imR)

K = 1.0e+03 * np.array([[2.3940, -0.0000,    0.9324],
                        [     0,  2.3981,    0.6283],
                        [     0,       0,    0.0010]])


plt.figure(0,figsize = (10, 4))
ax81 = plt.subplot(121)
plt.imshow(imL)
ax82 = plt.subplot(122)
plt.imshow(imR)
plt.show()

In [ ]:
# NOTE: corner_peaks and many other feature extraction functions return point coordinates as (y,x), that is (rows,cols)
keypointsL = corner_peaks(corner_harris(imLgray), threshold_rel=0.001, min_distance=15)
keypointsR = corner_peaks(corner_harris(imRgray), threshold_rel=0.001, min_distance=15)


print ('the number of features in images 1 and 2 are {:5d} and {:5d}'.format(keypointsL.shape[0],keypointsR.shape[0]))

fig = plt.figure(1,figsize = (10, 4))
axA = plt.subplot(111)
plt.gray()
matchesLR = np.empty((0,2))
plot_matches(axA, imL, imR, keypointsL, keypointsR, matchesLR)
axA.axis('off')

plt.show()

In [ ]:
# %matplotlib notebook

import numpy as np
import numpy.linalg as la
import matplotlib
import matplotlib.image as image
import matplotlib.pyplot as plt
from skimage.feature import (corner_harris, corner_peaks, plot_matches, BRIEF, match_descriptors)
from skimage.transform import warp, ProjectiveTransform, EssentialMatrixTransform, FundamentalMatrixTransform
from skimage.color import rgb2gray
from skimage.measure import ransac

# Indicate (E) inlier matches in image 1 and image 2
# loading two images (two camera views) and the corresponding matrix K (intrinsic parameters)
imL = image.imread("images/kronan1.jpg")
imR = image.imread("images/kronan2.jpg")
imLgray = rgb2gray(imL)
imRgray = rgb2gray(imR)

K = 1.0e+03 * np.array([[2.3940, -0.0000,    0.9324],
                        [     0,  2.3981,    0.6283],
                        [     0,       0,    0.0010]])


plt.figure(0,figsize = (10, 4))
ax81 = plt.subplot(121)
plt.imshow(imL)
ax82 = plt.subplot(122)
plt.imshow(imR)
plt.show()

In [ ]:
extractor = BRIEF()

extractor.extract(imLgray, keypointsL)
keypointsL = keypointsL[extractor.mask]         
descriptorsL = extractor.descriptors

extractor.extract(imRgray, keypointsR)
keypointsR = keypointsR[extractor.mask]
descriptorsR = extractor.descriptors

matchesLR = match_descriptors(descriptorsL, descriptorsR, cross_check=True)

print ('the number of matches is {:2d}'.format(matchesLR.shape[0]))

fig = plt.figure(2,figsize = (10, 4))
axA = plt.subplot(111)
axA.set_title("matches")
plt.gray()
plot_matches(axA, imL, imR, keypointsL, keypointsR, matchesLR) #, matches_color = 'r')
axA.axis('off')

plt.show()

In [ ]:

for i in range(1, len(num_images)):
    
    K = centered_calibration_matrices[i]]
    R = centered_rotation_matrices[i]]
    T = centered_translation_vectors[i]]
    
#     cross product matrix of T
#     T_x = np.zeros((3,3):
#     T_x[0] = ...
                   
    E = T_x @ R
    Ue,Se,Ve = la.svd(E)
    print (Se)
    
    # la.det(np.dot(Ue, Ve)) = 1 ==> switch sign of last col in Ve
    Ve_1 = (Ve.T @ np.diag(np.array([1,1,-1]))).T

    #Define W matrix
    W = np.array([
        [0, -1, 0],
        [1, 0, 0],
        [0, 0, 1]
    ])

    R1 = Ue @ W @ Ve_1
    R2 = Ue @ W.T @ Ve_1
    T1 = Ue[:, 2].reshape((3,1))
    T2 = -Ue[:, 2].reshape((3,1))



    # first camera matrix
    Pw = np.hstack([np.identity(n=3), np.zeros(shape=(3,1))])

    # four possible matrices for the second camera
    Pa = np.hstack([R1, T1])
    Pb = np.hstack([R1, T2])
    Pc = np.hstack([R2, T1])
    Pd = np.hstack([R2, T2])
    


    # la.det(np.dot(Ue, Ve)) = 1 ==> switch sign of last col in Ve
    Ve_1 = (Ve.T @ np.diag(np.array([1,1,-1]))).T

    #Define W matrix
    W = np.array([
        [0, -1, 0],
        [1, 0, 0],
        [0, 0, 1]
    ])

    R1 = Ue @ W @ Ve_1
    R2 = Ue @ W.T @ Ve_1
    T1 = Ue[:, 2].reshape((3,1))
    T2 = -Ue[:, 2].reshape((3,1))



    # first camera matrix
    Pw = np.hstack([np.identity(n=3), np.zeros(shape=(3,1))])

    # four possible matrices for the second camera
    Pa = np.hstack([R1, T1])
    Pb = np.hstack([R1, T2])
    Pc = np.hstack([R2, T1])
    Pd = np.hstack([R2, T2])